In [1]:
# loading the required packages
from qdrant_client import QdrantClient, models
import requests
from fastembed import TextEmbedding
import numpy as np
import json

In [2]:
# Embedding the query
embedding_model = TextEmbedding(model_name='jinaai/jina-embeddings-v2-small-en')
query = 'I just discovered the course. Can I still join it?'
query_vector = list(embedding_model.embed(query))
print(f'The length of the vector is {len(query_vector[0])}')
print(f'The minimal value of the resulting vector is: {round(min(query_vector[0]), 2)}')

The length of the vector is 512
The minimal value of the resulting vector is: -0.11


In [3]:
# Calculating the cosine similarity with another vector
doc = 'Can I still join the course after the start date?'
doc_vector = list(embedding_model.embed(doc))
cs = query_vector[0].dot(doc_vector[0])
print(f'The cosine similarity of the doc with the initial query is {round(cs, 1)}')

The cosine similarity of the doc with the initial query is 0.9


In [4]:
# Computing the embeddings for the text field and calculating the cosine between the query vector and all the documents
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

# Embed all documents
text_documents = [doc["text"] for doc in documents]
text_vectors = [np.array(vec) for vec in embedding_model.embed(text_documents)]
text_vectors = np.array(text_vectors)

# Matrix multiplication = all cosine similarities at once
cosine_similarities = text_vectors.dot(np.array(list(embedding_model.embed(query))[0]).reshape(-1))

# Find best match
most_similar_index = int(np.argmax(cosine_similarities))
print(f"Most similar document index: {most_similar_index}")
print(f"All cosine similarities: {cosine_similarities}")

Most similar document index: 1
All cosine similarities: [0.76920352 0.83360986 0.78624887 0.69141368 0.71836814]


In [5]:
# Concatenate 'question' and 'text' for each document
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
full_texts_vectors = np.array([np.array(vec) for vec in embedding_model.embed(full_texts)])
full_texts_vectors = np.array(full_texts_vectors)


# Matrix multiplication = all cosine similarities at once
cosine_similarities = full_texts_vectors.dot(np.array(list(embedding_model.embed(query))[0]).reshape(-1))

# Find best match
most_similar_index = int(np.argmax(cosine_similarities))
print(f"Most similar document index: {most_similar_index}")
print(f"All cosine similarities: {cosine_similarities}")

Most similar document index: 0
All cosine similarities: [0.86033261 0.85745452 0.820505   0.75649533 0.80158869]


In [6]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [7]:
# Get the list of supported models
supported_models = TextEmbedding.list_supported_models()

# Find the minimum dimension
min_dim = min(model["dim"] for model in supported_models)

print(f"Minimum embedding dimensionality: {min_dim}")

Minimum embedding dimensionality: 384


In [8]:
texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

small_model = TextEmbedding(model_name='BAAI/bge-small-en')

doc_vectors = np.array([np.array(vec) for vec in small_model.embed(texts)])

client = QdrantClient("http://localhost:6333")

collection_name = "ml-zoomcamp-faq"

# Create the collection with specified vector parameters
if collection_name not in [col.name for col in client.get_collections().collections]:
    client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=min_dim,
            distance=models.Distance.COSINE
        )
    )

In [9]:
points = [
    models.PointStruct(id=i, vector=doc_vectors[i].tolist(), payload={"question": documents[i]['question'], "text": documents[i]['text']})
    for i in range(len(doc_vectors))
]

In [11]:
client.upsert(collection_name=collection_name, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [12]:
search_result = client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=query,
        model='BAAI/bge-small-en'
    ),
    limit=1,
    with_payload=True
)
top_score = search_result.points[0].score
print(f"Highest score for the top result: {round(top_score, 2)}")

Highest score for the top result: 0.89
